In [86]:
import numpy as np
from pathlib import Path
from typing import List, Dict
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
import torch
from transformers import AutoModelForSequenceClassification
import accelerate
import transformers
from transformers import DistilBertConfig, DistilBertModel, DistilBertTokenizerFast
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from typing import Tuple
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import plotly.express as px
import re
import spacy

from enum import Enum
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import numpy as np
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib as plt
import zipfile
import requests
import io
import os


In [24]:
class Label(Enum):
    DOG = "dog"
    BIKE = "bike"
    BALL = "ball"
    WATER = "water"

ALL_LABELS = ["dog", "bike", "ball", "water"]




In [53]:
dataset_dir = Path("../data/final_dataset")
metadata_path = Path(dataset_dir / "metadata.csv")

metadata = pd.read_csv(metadata_path)
metadata.head()

,image_path,label,caption
0,new_ball_683.jpg,Label.BALL,"Two black dogs, their fur fluffed up in agitat..."
1,new_ball_683_spatial.jpg,Label.BALL,"Two black dogs, their fur fluffed up in agitat..."
2,new_ball_683_color.jpg,Label.BALL,"Two black dogs, their fur fluffed up in agitat..."
3,new_ball_196.jpg,Label.BALL,"In the sun-drenched backyard, a joyful black a..."
4,new_ball_196_spatial.jpg,Label.BALL,"In the sun-drenched backyard, a joyful black a..."


In [54]:
metadata["label"] = metadata["label"].apply(lambda x: x.split(".")[-1].lower())
metadata.head()

,image_path,label,caption
0,new_ball_683.jpg,ball,"Two black dogs, their fur fluffed up in agitat..."
1,new_ball_683_spatial.jpg,ball,"Two black dogs, their fur fluffed up in agitat..."
2,new_ball_683_color.jpg,ball,"Two black dogs, their fur fluffed up in agitat..."
3,new_ball_196.jpg,ball,"In the sun-drenched backyard, a joyful black a..."
4,new_ball_196_spatial.jpg,ball,"In the sun-drenched backyard, a joyful black a..."


In [55]:
metadata["label_in_caption"] = metadata.apply(
    lambda row: row["label"].lower() in row["caption"].lower(),
    axis=1
)
metadata.head()


,image_path,label,caption,label_in_caption
0,new_ball_683.jpg,ball,"Two black dogs, their fur fluffed up in agitat...",True
1,new_ball_683_spatial.jpg,ball,"Two black dogs, their fur fluffed up in agitat...",True
2,new_ball_683_color.jpg,ball,"Two black dogs, their fur fluffed up in agitat...",True
3,new_ball_196.jpg,ball,"In the sun-drenched backyard, a joyful black a...",True
4,new_ball_196_spatial.jpg,ball,"In the sun-drenched backyard, a joyful black a...",True


In [56]:
for l in ALL_LABELS:
    metadata[f"caption_has_{l}"] = (
        metadata["caption"]
        .str.lower()
        .str.contains(rf"\b{l}\b", regex=True, na=False)
    )

metadata.head()

,image_path,label,caption,label_in_caption,caption_has_dog,caption_has_bike,caption_has_ball,caption_has_water
0,new_ball_683.jpg,ball,"Two black dogs, their fur fluffed up in agitat...",True,False,False,True,False
1,new_ball_683_spatial.jpg,ball,"Two black dogs, their fur fluffed up in agitat...",True,False,False,True,False
2,new_ball_683_color.jpg,ball,"Two black dogs, their fur fluffed up in agitat...",True,False,False,True,False
3,new_ball_196.jpg,ball,"In the sun-drenched backyard, a joyful black a...",True,True,False,True,False
4,new_ball_196_spatial.jpg,ball,"In the sun-drenched backyard, a joyful black a...",True,True,False,True,False


In [57]:
df_unique = metadata.drop_duplicates(subset=["caption"])
rows = []

for lbl in Label:
    col = f"caption_has_{lbl.value}"
    rows.append({
        "label": lbl.value,
        "true_count": int(df_unique[col].sum()),
        "false_count": int((~df_unique[col]).sum()),
        "total_unique_captions": len(df_unique)
    })


stats_df = pd.DataFrame(rows)
stats_df


,label,true_count,false_count,total_unique_captions
0,dog,510,890,1400
1,bike,192,1208,1400
2,ball,272,1128,1400
3,water,280,1120,1400


In [64]:
missing = df_unique[~df_unique["label_in_caption"]]

repartition = (
    missing["label"]
    .value_counts()
    .reset_index(name="count")
    .rename(columns={"index": "label"})
)
repartition["pct"] = repartition["count"] / repartition["count"].sum()
by_class = df_unique.groupby("label")["label_in_caption"].agg(
    total="size",
    ok="sum"
).reset_index()

by_class["missing"] = by_class["total"] - by_class["ok"]
by_class["missing_rate"] = by_class["missing"] / by_class["total"]

print(by_class)


   label  total   ok  missing  missing_rate
0   ball    350  316       34      0.097143
1   bike    350  241      109      0.311429
2    dog    350  335       15      0.042857
3  water    350  252       98      0.280000


In [66]:
missing[["image_path", "label", "caption"]].head(5)


,image_path,label,caption
600,base_water_009.jpg,water,a boy cleans the bubbles off his face .
603,base_water_091.jpg,water,A male and a female are leaning against railin...
615,base_water_129.jpg,water,a man jumping in a mud puddle in the middle of...
618,base_water_137.jpg,water,A boy gets flipped into the river by another b...
624,base_water_066.jpg,water,A kayaker goes through the waves holding his p...


In [70]:
fig = px.bar(
    repartition.sort_values("count", ascending=False),
    x="label",
    y="count",
    text="count",
    title="Captions uniques où le label attendu n'apparaît pas"
)
fig.update_traces(textposition="outside")
fig.update_layout(yaxis_title="Nombre de captions uniques", xaxis_title="Label")
fig.show()

### SAUVEGARDE CAPTION PROBLEMATIQUE (n'ayant pas leur label dans la caption ou/et un autre mot "label" dans la caption)

In [76]:
def other_labels(row):
    found = []
    for lbl in Label:
        v = lbl.value
        if row.get(f"caption_has_{v}", False) and row["label"] != v:
            found.append(v)
    return found
problem = df_unique[~df_unique["label_in_caption"]].copy()
problem["other_labels_in_caption"] = problem.apply(other_labels, axis=1)
cols = [
    "image_path",
    "label",
    "caption",
    "other_labels_in_caption"
]
total_captions = len(df_unique)
problem_captions = len(problem)

ratio = problem_captions / total_captions
percent = ratio * 100

print(f"TOTAL CAPTION: {total_captions} \n PROBLEM CAPTION: {problem_captions} \n REP.: {int(percent)}%")

export_df = problem[cols]
export_df.to_csv(
    dataset_dir /"problematic_captions.csv",
    index=False,
    encoding="utf-8"
)



TOTAL CAPTION: 1400 
 PROBLEM CAPTION: 256 
 REP.: 18%


### ETUDE CORRELATION LABEL DANS CAPTION ET LABEL REEL

In [59]:
results = []

for lbl in Label:
    v = lbl.value
    col = f"caption_has_{v}"

    df_fp = df_unique[
        (df_unique[col]) &
        (df_unique["label"] != v)
    ]

    counts = df_fp["label"].value_counts()

    for real_label, count in counts.items():
        results.append({
            "caption_label": v,
            "real_label": real_label,
            "count": count
        })

real_label_df = pd.DataFrame(results)
pivot = real_label_df.pivot_table(
    index="caption_label",
    columns="real_label",
    values="count",
    fill_value=0
)
pivot

real_label,ball,bike,dog,water
caption_label,,,,
ball,0.0,0.0,27.0,4.0
bike,0.0,0.0,0.0,1.0
dog,120.0,6.0,0.0,74.0
water,21.0,10.0,24.0,0.0


In [35]:
'''
quand une caption mentionne x, à quel point le vrai label est souvent Y?
quand une caption mentionne ball, à quel point le vrai label est souvent dog?
'''


corr_score = pivot.div(pivot.sum(axis=1), axis=0)
print(corr_score)

real_label         ball      bike       dog     water
caption_label                                        
ball           0.000000  0.000000  0.870968  0.129032
bike           0.000000  0.000000  0.000000  1.000000
dog            0.600000  0.030000  0.000000  0.370000
water          0.381818  0.181818  0.436364  0.000000


100% des captions ayant bike sont labelisés water? -> pivot table calculée sur les exemples où la caption contient un mot label qui n'est pas le label réel donc cela veut dire que 100% des captions qui ont bike sans être labellisée bike étaient labellisée water

### NOMBRE DE MOTS PAR CAPTION

In [84]:
df_unique = metadata.drop_duplicates(subset=["caption"]).copy()


def word_count(text):
    if not isinstance(text, str):
        return 0
    # mots = séquences de lettres/chiffres
    return len(re.findall(r"\b\w+\b", text))

df_unique["word_count"] = df_unique["caption"].apply(word_count)
stats_words_by_label = (
    df_unique
    .groupby("label")["word_count"]
    .agg(
        mean="mean",
        median="median",
        std="std",
        min="min",
        max="max",
        q25=lambda x: x.quantile(0.25),
        q75=lambda x: x.quantile(0.75),
    )
    .reset_index()
)
stats_words_by_label


,label,mean,median,std,min,max,q25,q75
0,ball,47.605714,65.0,32.418803,3,107,12.0,75.00
1,bike,45.697143,61.0,31.179115,4,98,11.0,73.00
2,dog,46.942857,63.0,31.726032,5,104,12.0,75.75
3,water,45.865714,61.0,30.842547,4,100,12.0,73.00


### LEMMATISATION ET RECHERCHE DE THEMES

In [93]:
LABEL_TEXT = {
    "dog":   "a dog, a puppy, a canine animal",
    "bike":  "a bicycle, a bike, cycling, a cyclist riding",
    "ball":  "a ball, playing with a ball, a toy ball",
    "water": "water, a lake, a river, the sea, ocean, beach"
}


In [129]:
import numpy as np
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2") 

labels = list(LABEL_TEXT.keys())
label_emb = model.encode([LABEL_TEXT[l] for l in labels], normalize_embeddings=True)

captions = df_unique["caption"].fillna("").astype(str).tolist()
cap_emb = model.encode(captions, normalize_embeddings=True)

# similarité
sim = cap_emb @ label_emb.T   #(n_captions, n_labels)


In [130]:
best_idx = sim.argmax(axis=1)
df_unique["theme_pred_label"] = [labels[i] for i in best_idx]

df_unique["theme_matches_label"] = df_unique["theme_pred_label"] == df_unique["label"]

df_unique.loc[
    df_unique["theme_pred_label"] != df_unique["label"],
    ["image_path", "label", "caption", "theme_pred_label"]
].head(5)


,image_path,label,caption,theme_pred_label
15,new_ball_241.jpg,ball,"A serene lake setting, where the tranquil atmo...",water
87,new_ball_623.jpg,ball,The sun beats down on the sandy beach as three...,water
180,new_ball_675.jpg,ball,"A playful brown dog, its fur fluffed up from t...",water
207,new_ball_279.jpg,ball,A playful brown dog is seen splashing in a cal...,dog
270,new_ball_399.jpg,ball,The black dog's ears flapped in the sea breeze...,water


In [131]:
df_unique["theme_score_true"] = [
    sim[i, labels.index(df_unique.iloc[i]["label"])] for i in range(len(df_unique))
]
df_unique["theme_score_best"] = sim.max(axis=1)
df_unique["theme_margin"] = df_unique["theme_score_best"] - df_unique["theme_score_true"]


In [132]:
df_unique.groupby("label")["theme_matches_label"].mean()
df_unique.loc[~df_unique["theme_matches_label"], ["label", "caption", "theme_pred_label"]].head(20)
df_unique.head(5)

,image_path,label,caption,label_in_caption,caption_has_dog,caption_has_bike,caption_has_ball,caption_has_water,word_count,lemmas,theme_pred_label,theme_matches_label,theme_score_true,theme_score_best,theme_margin
0,new_ball_683.jpg,ball,"Two black dogs, their fur fluffed up in agitat...",True,False,False,True,False,61,"{war, game, leaf, soccer, intense, tug, fur, s...",ball,True,0.438486,0.438486,0.0
3,new_ball_196.jpg,ball,"In the sun-drenched backyard, a joyful black a...",True,True,False,True,False,75,"{leap, drench, backyard, soccer, afternoon, te...",ball,True,0.509551,0.509551,0.0
6,new_ball_092.jpg,ball,"A young boy, his eyes fixed on the ball in his...",True,True,False,True,False,76,"{tumble, calm, charge, run, playfully, heap, y...",ball,True,0.434281,0.434281,0.0
9,new_ball_708.jpg,ball,"A sunny day at the local park, where families ...",True,False,False,False,False,66,"{day, sport, practice, laughter, gather, child...",ball,True,0.380039,0.380039,0.0
12,new_ball_190.jpg,ball,A young boy stands in front of the bathroom si...,True,False,False,True,False,58,"{t, add, mundane, hand, young, contrast, surro...",ball,True,0.210201,0.210201,0.0


In [133]:
nlp = spacy.load("en_core_web_sm")

def roles_of_label_in_caption(caption: str, label: str):
    if not isinstance(caption, str) or not caption.strip():
        return []
    doc = nlp(caption)
    label_l = label.lower()

    out = []
    for tok in doc:
        # match mot entier (évite ball dans football)
        if tok.text.lower() == label_l:
            out.append({
                "match": tok.text,
                "pos": tok.pos_,          # ex: NOUN, VERB
                "tag": tok.tag_,          # ex: NN, VBP (plus fin)
                "dep": tok.dep_,          # ex: nsubj, dobj, pobj, compound...
                "head": tok.head.text,    # mot "chef"
                "head_pos": tok.head.pos_,
                "span": doc[max(0, tok.i-3): tok.i+4].text  # mini-contexte
            })
    return out


In [134]:
df_ok = df_unique[df_unique["label_in_caption"]].copy()

df_ok["label_roles"] = df_ok.apply(
    lambda row: roles_of_label_in_caption(row["caption"], row["label"]),
    axis=1
)


In [113]:
df_ok.loc[:, ["image_path", "label_roles"]].head(10)


,image_path,label_roles
0,new_ball_683.jpg,"[{'match': 'ball', 'pos': 'NOUN', 'tag': 'NN',..."
3,new_ball_196.jpg,"[{'match': 'ball', 'pos': 'NOUN', 'tag': 'NN',..."
6,new_ball_092.jpg,"[{'match': 'ball', 'pos': 'NOUN', 'tag': 'NN',..."
9,new_ball_708.jpg,[]
12,new_ball_190.jpg,"[{'match': 'ball', 'pos': 'NOUN', 'tag': 'NN',..."
15,new_ball_241.jpg,"[{'match': 'ball', 'pos': 'NOUN', 'tag': 'NN',..."
18,new_ball_446.jpg,"[{'match': 'ball', 'pos': 'NOUN', 'tag': 'NN',..."
21,new_ball_700.jpg,"[{'match': 'ball', 'pos': 'NOUN', 'tag': 'NN',..."
24,new_ball_002.jpg,"[{'match': 'ball', 'pos': 'NOUN', 'tag': 'NN',..."
27,new_ball_425.jpg,"[{'match': 'ball', 'pos': 'NOUN', 'tag': 'NN',..."


pobj -> objet d'une préposition \
dobj -> objet direct


In [115]:
rows = []
for _, r in df_ok.iterrows():
    for occ in r["label_roles"]:
        rows.append({
            "label": r["label"],
            "caption": r["caption"],
            **occ
        })

roles_df = pd.DataFrame(rows)
roles_df.head(10)

dep_stats = (
    roles_df.groupby(["label", "pos"])
    .size()
    .reset_index(name="count")
    .sort_values(["label", "count"], ascending=[True, False])
)
dep_stats.head(10)


,label,pos,count
0,ball,NOUN,363
1,ball,PROPN,3
2,bike,NOUN,227
3,dog,NOUN,441
4,dog,PROPN,2
6,water,NOUN,296
5,water,ADJ,1
7,water,PROPN,1


In [116]:
nlp = spacy.load("en_core_web_sm")

DEP_TO_ROLE_FR = {
    "nsubj": "sujet",
    "nsubjpass": "sujet (voix passive)",
    "obj": "objet",
    "dobj": "objet direct",
    "iobj": "objet indirect",
    "pobj": "complément d’une préposition (ex: 'in water')",
    "attr": "attribut (ex: 'it is a dog')",
    "appos": "apposition (ex: 'a toy, a ball')",
    "compound": "nom composé (ex: 'ball game')",
    "poss": "possessif (ex: \"dog's ball\")",
    "conj": "élément coordonné (ex: 'dog and ball')",
    "amod": "modificateur adjectival",
    "npadvmod": "complément circonstanciel en groupe nominal",
}

POS_TO_FR = {
    "NOUN": "nom",
    "PROPN": "nom propre",
    "VERB": "verbe",
    "AUX": "auxiliaire",
    "ADJ": "adjectif",
    "ADV": "adverbe",
    "ADP": "préposition",
}

def label_role_in_caption(caption: str, label: str):
    """Retourne le rôle du label dans la caption (première occurrence)."""
    if not isinstance(caption, str) or not caption.strip():
        return None

    doc = nlp(caption)
    lab = label.lower()

    for tok in doc:
        if tok.text.lower() == lab:
            return {
                "match": tok.text,
                "pos": tok.pos_,
                "pos_fr": POS_TO_FR.get(tok.pos_, tok.pos_),
                "dep": tok.dep_,
                "role_fr": DEP_TO_ROLE_FR.get(tok.dep_, tok.dep_),
                "head": tok.head.text,
            }
    return None


In [117]:
df_ok = df_unique[df_unique["label_in_caption"]].copy()

df_ok["label_role"] = df_ok.apply(
    lambda r: label_role_in_caption(r["caption"], r["label"]),
    axis=1
)


In [118]:
df_ok_view = df_ok.loc[:, ["label", "caption", "label_role"]]
df_ok_view.head(10)


,label,caption,label_role
0,ball,"Two black dogs, their fur fluffed up in agitat...","{'match': 'ball', 'pos': 'NOUN', 'pos_fr': 'no..."
3,ball,"In the sun-drenched backyard, a joyful black a...","{'match': 'ball', 'pos': 'NOUN', 'pos_fr': 'no..."
6,ball,"A young boy, his eyes fixed on the ball in his...","{'match': 'ball', 'pos': 'NOUN', 'pos_fr': 'no..."
9,ball,"A sunny day at the local park, where families ...",None
12,ball,A young boy stands in front of the bathroom si...,"{'match': 'ball', 'pos': 'NOUN', 'pos_fr': 'no..."
15,ball,"A serene lake setting, where the tranquil atmo...","{'match': 'ball', 'pos': 'NOUN', 'pos_fr': 'no..."
18,ball,The sandy brown dog was running around in circ...,"{'match': 'ball', 'pos': 'NOUN', 'pos_fr': 'no..."
21,ball,The little girl runs through the lush green pa...,"{'match': 'ball', 'pos': 'NOUN', 'pos_fr': 'no..."
24,ball,"A sunny afternoon in a lush green field, with ...","{'match': 'ball', 'pos': 'NOUN', 'pos_fr': 'no..."
27,ball,A soccer player in a green jersey has just sto...,"{'match': 'ball', 'pos': 'NOUN', 'pos_fr': 'no..."


In [119]:
df_ok["label_pos"] = df_ok["label_role"].apply(lambda x: x["pos_fr"] if x else None)
df_ok["label_role_fr"] = df_ok["label_role"].apply(lambda x: x["role_fr"] if x else None)
df_ok["label_head"] = df_ok["label_role"].apply(lambda x: x["head"] if x else None)

df_ok.loc[:, ["label", "caption", "label_pos", "label_role_fr", "label_head"]].head(10)


,label,caption,label_pos,label_role_fr,label_head
0,ball,"Two black dogs, their fur fluffed up in agitat...",nom,complément d’une préposition (ex: 'in water'),over
3,ball,"In the sun-drenched backyard, a joyful black a...",nom,objet direct,catch
6,ball,"A young boy, his eyes fixed on the ball in his...",nom,complément d’une préposition (ex: 'in water'),on
9,ball,"A sunny day at the local park, where families ...",None,None,None
12,ball,A young boy stands in front of the bathroom si...,nom,objet direct,holds
15,ball,"A serene lake setting, where the tranquil atmo...",nom,complément d’une préposition (ex: 'in water'),on
18,ball,The sandy brown dog was running around in circ...,nom,complément d’une préposition (ex: 'in water'),after
21,ball,The little girl runs through the lush green pa...,nom,complément d’une préposition (ex: 'in water'),after
24,ball,"A sunny afternoon in a lush green field, with ...",nom,complément d’une préposition (ex: 'in water'),on
27,ball,A soccer player in a green jersey has just sto...,nom,objet direct,stolen


In [120]:
role_by_label = (
    df_ok.groupby(["label", "label_role_fr"])
    .size()
    .reset_index(name="count")
    .sort_values(["label", "count"], ascending=[True, False])
)
role_by_label

,label,label_role_fr,count
3,ball,complément d’une préposition (ex: 'in water'),119
5,ball,objet direct,93
7,ball,sujet,16
1,ball,"apposition (ex: 'a toy, a ball')",4
4,ball,nom composé (ex: 'ball game'),4
0,ball,ROOT,1
2,ball,complément circonstanciel en groupe nominal,1
6,ball,"possessif (ex: ""dog's ball"")",1
8,ball,sujet (voix passive),1
9,ball,élément coordonné (ex: 'dog and ball'),1


In [122]:
role_by_label = (
    df_ok.groupby(["label", "label_role_fr"])
    .size()
    .reset_index(name="count")
)

fig = px.bar(
    role_by_label,
    x="label_role_fr",
    y="count",
    color="label",
    barmode="group",
    title="Distribution des rôles syntaxiques par label"
)

fig.update_layout(
    xaxis_title="Rôle syntaxique",
    yaxis_title="Nombre d’occurrences",
    xaxis_tickangle=-30
)

fig.show()
